1. Label data using Named Entity Recognition
2. Split data into train and test
3. train model
4. test model

In [1]:
import pandas as pd

import spacy
from spacy.matcher import PhraseMatcher

from scholarly import scholarly

import requests
import mimetypes
import os

import uuid

import PyPDF2

from PyPDF2 import PdfReader

In [2]:
ROOT_FOLDER = '../../../'

In [3]:
DATA_FOLDER = '../../../data/'

In [4]:
DETECT_ML_MODEL_FILES_FOLDER = '../../../detect_ml_model_files/'

In [5]:
data_df = pd.read_parquet(f'{DATA_FOLDER}detect_ml_models.parquet')

In [6]:
# drop records where the file_name is Null
data_df = data_df.dropna()

In [7]:
data_df

,title,abstract,year,url,author_id,query,file_name,file_path
0,Linear regression,Linear regression plays a fundamental role in ...,2012,https://www.cs.columbia.edu/~djhsu/coms4771-f2...,"[wcgZZP8AAAAJ, , El-UNYoAAAAJ]",Linear Regression,7cedd2b0-ac03-49fb-9ef1-398b1a5d7fe1.pdf,../../../detect_ml_model_files/7cedd2b0-ac03-4...
1,Introduction to linear regression analysis,Equation (1.2) is called a linear regression m...,2021,http://sutlib2.sut.ac.th/sut_contents/H133678.pdf,"[5PboKNAAAAAJ, , y2eb6cQAAAAJ]",Linear Regression,91374318-7135-4f51-a3bc-720ececd88de.pdf,../../../detect_ml_model_files/91374318-7135-4...
2,Linear regression,"linear regression, a very simple approach for ...",2023,https://datamineaz.org/readings/ISL_chp3.pdf,"[KUIjZqgAAAAJ, bHZf-c8AAAAJ, tQVe-fAAAAAJ, ZpG...",Linear Regression,6d3627b9-6b0b-423c-8898-3185837a7617.pdf,../../../detect_ml_model_files/6d3627b9-6b0b-4...
4,Applied linear regression,Applied linear regression Applied linear regre...,2005,https://www.stat.cmu.edu/~brian/valerie/617-20...,[wlu6jZQAAAAJ],Linear Regression,c7fed73c-8e13-4770-a589-67708aab0b7e.pdf,../../../detect_ml_model_files/c7fed73c-8e13-4...
8,A review on linear regression comprehensive in...,We discuss linear regression and polynomial re...,2020,https://jastt.org/index.php/jasttpath/article/...,"[G9U01kwAAAAJ, aBdgHxkAAAAJ]",Linear Regression,f8d408d7-9c86-4d72-b6af-6538ed46970f.pdf,../../../detect_ml_model_files/f8d408d7-9c86-4...
...,...,...,...,...,...,...,...,...
1227,BERT: a review of applications in natural lang...,"relative to the original BERT model, which is ...",2021,https://arxiv.org/pdf/2103.11943,[],BERT,91ebf298-65bd-4429-8c60-b17cb80306fe.pdf,../../../detect_ml_model_files/91ebf298-65bd-4...
1228,What does BERT learn about the structure of la...,language structure learned by BERT. We first s...,2019,https://inria.hal.science/hal-02131630/document,"[X7SMP1EAAAAJ, HXUT9ZkAAAAJ, P7EtARsAAAAJ]",BERT,65f04db1-ad62-4f48-9e80-3ae5a11bf147.pdf,../../../detect_ml_model_files/65f04db1-ad62-4...
1229,Visualizing and understanding the effectivenes...,trajectories of fine-tuning BERT on specific d...,2019,https://arxiv.org/pdf/1908.05620,"[cqOLO7IAAAAJ, wEfQgPgAAAAJ, G-V1VpwAAAAJ, ]",BERT,03d43387-8413-413c-8ce8-92b3512cedfb.pdf,../../../detect_ml_model_files/03d43387-8413-4...
1230,A comprehensive survey on pretrained foundatio...,Pretrained Foundation Models (PFMs) are regard...,2024,https://arxiv.org/pdf/2302.09419,"[HWx73DcAAAAJ, AHg-JGIAAAAJ, nPvWFpkAAAAJ, fh1...",BERT,a60911d8-b83b-4c2a-8555-2ac3b09b5025.pdf,../../../detect_ml_model_files/a60911d8-b83b-4...


The query column represents the machine learning model name that was used to search Google Scholar.

There should be five papers per model.

Add a new column to represent an identifier/order for the paper within each query

In [8]:
#data_df['counter'] = data_df.groupby('query').cumcount() + 1
data_df.loc[:, 'counter'] = data_df.groupby('query').cumcount() + 1

In [9]:
data_df

,title,abstract,year,url,author_id,query,file_name,file_path,counter
0,Linear regression,Linear regression plays a fundamental role in ...,2012,https://www.cs.columbia.edu/~djhsu/coms4771-f2...,"[wcgZZP8AAAAJ, , El-UNYoAAAAJ]",Linear Regression,7cedd2b0-ac03-49fb-9ef1-398b1a5d7fe1.pdf,../../../detect_ml_model_files/7cedd2b0-ac03-4...,1
1,Introduction to linear regression analysis,Equation (1.2) is called a linear regression m...,2021,http://sutlib2.sut.ac.th/sut_contents/H133678.pdf,"[5PboKNAAAAAJ, , y2eb6cQAAAAJ]",Linear Regression,91374318-7135-4f51-a3bc-720ececd88de.pdf,../../../detect_ml_model_files/91374318-7135-4...,2
2,Linear regression,"linear regression, a very simple approach for ...",2023,https://datamineaz.org/readings/ISL_chp3.pdf,"[KUIjZqgAAAAJ, bHZf-c8AAAAJ, tQVe-fAAAAAJ, ZpG...",Linear Regression,6d3627b9-6b0b-423c-8898-3185837a7617.pdf,../../../detect_ml_model_files/6d3627b9-6b0b-4...,3
4,Applied linear regression,Applied linear regression Applied linear regre...,2005,https://www.stat.cmu.edu/~brian/valerie/617-20...,[wlu6jZQAAAAJ],Linear Regression,c7fed73c-8e13-4770-a589-67708aab0b7e.pdf,../../../detect_ml_model_files/c7fed73c-8e13-4...,4
8,A review on linear regression comprehensive in...,We discuss linear regression and polynomial re...,2020,https://jastt.org/index.php/jasttpath/article/...,"[G9U01kwAAAAJ, aBdgHxkAAAAJ]",Linear Regression,f8d408d7-9c86-4d72-b6af-6538ed46970f.pdf,../../../detect_ml_model_files/f8d408d7-9c86-4...,5
...,...,...,...,...,...,...,...,...,...
1227,BERT: a review of applications in natural lang...,"relative to the original BERT model, which is ...",2021,https://arxiv.org/pdf/2103.11943,[],BERT,91ebf298-65bd-4429-8c60-b17cb80306fe.pdf,../../../detect_ml_model_files/91ebf298-65bd-4...,1
1228,What does BERT learn about the structure of la...,language structure learned by BERT. We first s...,2019,https://inria.hal.science/hal-02131630/document,"[X7SMP1EAAAAJ, HXUT9ZkAAAAJ, P7EtARsAAAAJ]",BERT,65f04db1-ad62-4f48-9e80-3ae5a11bf147.pdf,../../../detect_ml_model_files/65f04db1-ad62-4...,2
1229,Visualizing and understanding the effectivenes...,trajectories of fine-tuning BERT on specific d...,2019,https://arxiv.org/pdf/1908.05620,"[cqOLO7IAAAAJ, wEfQgPgAAAAJ, G-V1VpwAAAAJ, ]",BERT,03d43387-8413-413c-8ce8-92b3512cedfb.pdf,../../../detect_ml_model_files/03d43387-8413-4...,3
1230,A comprehensive survey on pretrained foundatio...,Pretrained Foundation Models (PFMs) are regard...,2024,https://arxiv.org/pdf/2302.09419,"[HWx73DcAAAAJ, AHg-JGIAAAAJ, nPvWFpkAAAAJ, fh1...",BERT,a60911d8-b83b-4c2a-8555-2ac3b09b5025.pdf,../../../detect_ml_model_files/a60911d8-b83b-4...,4


In [10]:
# nlp = spacy.blank("en")
# ruler = nlp.add_pipe("entity_ruler")

#nlp = spacy.load("en_core_web_sm")
#ruler = nlp.add_pipe("entity_ruler", before="ner")

nlp = spacy.load("en_core_web_sm", exclude=["ner"])  # Exclude the default NER component
ruler = nlp.add_pipe("entity_ruler", last=True)

# Load the EntityRuler patterns from a file
ruler.from_disk(f"{DATA_FOLDER}ml_entity_ruler_patterns")

Lets take the first four files for a model for training and the last one for test. This will give us a 80/20 split of files at least.

The query results from Google Scholar are in descending order of popularity. This should mean that the files are not necessarily in order of published date, which is good.

Further analysis is required to look at the terms in each file and create a more balanced training/test split.



In [11]:
training_files = data_df.query('counter < 5')['file_path'].tolist()

In [12]:
len(training_files)

396

In [13]:
test_files = data_df.query('counter == 5')['file_path'].tolist()

In [14]:
len(test_files)

99

There should be 4 times more training_files than test_files, lets test that

In [15]:
len(test_files) * 4 == len(training_files)

True

In [16]:
data_df.groupby(['query']).count().reset_index().query('counter != 5')['query'].values

array([], dtype=object)

In [17]:
# Test the pipeline
text = "Support Vector Machine, Support Vector Machines, SVM, and S.V.M. are popular machine learning methods."
doc = nlp(text)

# Print detected entities
for ent in doc.ents:
    print(ent.text, ent.label_)


Support Vector Machine ML_METHOD
Support Vector Machines ML_METHOD
SVM ML_METHOD


Find terms in file to use as labels

In [18]:
def create_labelled_docs(files):
    docs = list()
    for pdf_file_path in files:
        with open(pdf_file_path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page_num, page in enumerate(pdf_reader.pages):
                # Extract text from the page
                text = page.extract_text()
                doc = nlp(text)

                # Extract sentences with ML_METHOD entities
                filtered_sentences = [
                    sentence.text
                    for sentence in doc.sents
                    if any(ent.label_ == "ML_METHOD" for ent in sentence.ents)
                ]

                if len(filtered_sentences) > 0:
                    # Create a new text containing only those sentences
                    filtered_text = " ".join(filtered_sentences)
                    
                    # Create a new Doc object with the filtered text
                    filtered_doc = nlp(filtered_text)
                    docs.append(filtered_doc)
                    
    return docs

In [19]:
training_docs = create_labelled_docs(files=training_files)
if len(training_docs) < 1:
    print('There are no labels created for the requested files!!!')
else:
    # save it in a format spacy can use
    training_doc_bin = DocBin(docs=training_docs)
    training_doc_bin.to_disk("./train.spacy")


KeyboardInterrupt: 

In [15]:
def search_in_file(pdf_file_path):
    search_results = dict()
    # Extract text from PDF
    with open(pdf_file_path, "rb") as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        for page_num, page in enumerate(pdf_reader.pages):
            # Extract text from the page
            text = page.extract_text()
            if text:
                # Use spaCy to process the text
                doc = nlp(text)
                
                # Detect entities
                for ent in doc.ents:
                    if ent.label_ == "ML_METHOD":
                        if ent.text in search_results:
                            search_results[ent.text].append(page_num + 1)  # Store page number (1-indexed)
                        else:
                            search_results[ent.text] = [page_num + 1]  # Store page number (1-indexed)
    
    # Display search results
    for term, pages in search_results.items():
        if pages:
            print(f"'{term}' found on page(s): {set(pages)}")
        #else:
        #    print(f"'{term}' not found in the document.")

In [17]:
training_files

In [18]:
pdf_file_path = file_paths[0]

print(pdf_file_path)

../../detect_ml_model_files/18d9a07e-aaa5-4cbe-a094-ccc035a96392.pdf


In [19]:
with open(pdf_file_path, "rb") as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    for page_num, page in enumerate(pdf_reader.pages):
        # Extract text from the page
        text = page.extract_text()
        break

In [20]:
doc = nlp(text)

In [21]:
for ent in doc.ents:
    print(ent)

Linear regression
linear regression


In [22]:
#nlp.add_pipe("sentencizer", before="parser")

# Recreate the Doc object
#doc = nlp(text)

for sentence in doc.sents:
    print(sentence.text)

Linear regression
Daniel Hsu (COMS 4771)
Maximum likelihood estimation
One of the simplest linear regression models is the following: (X1,Y1),..., (Xn,Yn),(X,Y)are iid random
pairs taking values in Rd×R, and
Y|X=x∼N(xTw,σ2),x∈Rd.

Here, the vector w∈Rdand scalarσ2>0are the parameters of the model.
(The marginal distribution of
Xis unspeciﬁed.)

Thelog-likelihood of(w,σ2)given (Xi,Yi)
= (xi,yi)fori= 1,...,nis
n/summationdisplay
i=1/braceleftBigg
ln1√
2πσ2−(yi−xT
iw)2
2σ2/bracerightBigg
+T,
whereTis some quantity that does not depend on (w,σ2).
Therefore, maximizing the log-likelihood over
w∈Rd(for anyσ2>0) is the same as minimizing
1
nn/summationdisplay
i=1(xT
iw−yi)2.

So, the maximum likelihood estimator (MLE) ofwin this model is
ˆw∈arg min
w∈Rd1
nn/summationdisplay
i=1(xT
iw−yi)2.

(It is not necessarily uniquely determined.)

Empirical risk minimization
LetPnbe the empirical distribution on(x1,y1),..., (xn,yn)∈Rd×R, i.e., the probability distribution over
Rd×Rwith probability mass f

In [23]:
from spacy import displacy

# Render the entities in a Jupyter Notebook or as HTML
# displacy.render(doc, style="ent", jupyter=True)  # Use jupyter=True if in a notebook

options = {"colors": {"ORG": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "ML_METHOD": "#f6c5be"}}
displacy.render(doc, style="ent", options=options, jupyter=True)


Lets only select sentences with a ML_METHOD entity


In [24]:
# Extract sentences with ML_METHOD entities
filtered_sentences = [
    sentence.text
    for sentence in doc.sents
    if any(ent.label_ == "ML_METHOD" for ent in sentence.ents)
]

# Create a new text containing only those sentences
filtered_text = " ".join(filtered_sentences)

# Create a new Doc object with the filtered text
filtered_doc = nlp(filtered_text)

# Print the filtered sentences
print("Filtered Sentences:")
for sentence in filtered_doc.sents:
    print(sentence.text)

Filtered Sentences:
Linear regression
Daniel Hsu (COMS 4771)
Maximum likelihood estimation
One of the simplest linear regression models is the following: (X1,Y1),..., (Xn,Yn),(X,Y)are iid random
pairs taking values in Rd×R, and
Y|X=x∼N(xTw,σ2),x∈Rd.



In [25]:
#displacy.render(filtered_doc, style="ent", jupyter=True)  # Use jupyter=True if in a notebook

options = {"colors": {"ML_METHOD": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}}
displacy.render(filtered_doc, style="ent", options=options, jupyter=True)